In [120]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score # Avaliar o r2 do modelo
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder # Converter os dados categoricos em numericos
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import GridSearchCV # Testar os melhores parametros
from xgboost import XGBRegressor # Modelo para Regressao
import pandas as pd
from math import sqrt
    
from IPython.core.pylabtools import figsize
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [153]:
df = pd.read_pickle('vendas.pickle')

In [154]:
df.drop('vl_total', 1, inplace=True)
y_col = 'qtd'

In [204]:
y = df[y_col].to_numpy()
X = df[df.columns.drop(y_col)].to_numpy()

In [205]:
valid_size = 100

y = y[:-valid_size]
X = X[:-valid_size]
y_valid = y[-valid_size:]
X_valid = X[-valid_size:]

In [206]:
X.shape, y.shape, X_valid.shape, y_valid.shape

((10027, 15), (10027,), (100, 15), (100,))

In [210]:
1027/7

146.71428571428572

In [225]:
n_splits = 20
tscv = TimeSeriesSplit(n_splits)

In [226]:
from sklearn.linear_model import SGDRegressor

In [227]:
model = SGDRegressor(loss='squared_loss')

min_max_scaler = MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)
X_valid_scaled = min_max_scaler.fit_transform(X_valid)

for train_index, test_index in tscv.split(X):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    rmse = (sqrt(mean_squared_error(y_test, y_pred_test)))
    print(rmse)
    
sgdr = model

12.480980480348473
9.033907580844488
9.15045739260311
14.841602252222911
9.728283438567331
19.020915327749826
13.965302181234458
11.211551363852617
8.494287703646528
6.717103550778346
7.337803443171452
8.813537265586568
15.015636935590662
15.771381211340207
15.946300891096394
15.877500888613884
31.534496910986174
14.706855541248668
13.911718023648628
16.47095646660485


In [228]:
from sklearn.ensemble import RandomForestRegressor

In [229]:
model = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0)

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    rmse = (sqrt(mean_squared_error(y_test, y_pred_test)))
    print(rmse)
    
rfr = model

9.753054413466252
13.556678507324108
6.918761353044153
12.104176235569236
8.47118862220385
13.6424538674758
11.962761650825673
8.506353902764543
7.461428704502173
5.044706482725764
6.078186418526894
6.215582142235457
10.79585615034157
19.20734063607642
12.402027041071282
12.408891707175917
19.983633260688457
14.155900661740791
10.277738131137765
15.163587437642704


In [ ]:
# A parameter grid for XGBoost
params = {
    'min_child_weight': [4, 5],
    'gamma': [i / 10.0 for i in range(3, 6)],
    'subsample': [i / 10.0 for i in range(6, 11)],
    'colsample_bytree': [i / 10.0 for i in range(6, 11)],
    'max_depth': [3, 4, 5, 6]
}

# Initialize XGB and GridSearch
xgb_reg = XGBRegressor(nthread=-1, objective='reg:squarederror')
grid = GridSearchCV(xgb_reg, params)
grid.fit(X_train, y_train)
gridcv_xgb = grid.best_estimator_
print(r2_score(y_test, gridcv_xgb.predict(X_test)))

In [ ]:
gridcv_xgb

In [208]:
#model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)

In [218]:
model = gridcv_xgb

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    rmse = (sqrt(mean_squared_error(y_test, y_pred_test)))
    print(rmse)
    
xgb = model

15.945480872625653
14.610703160343677
14.706269696772003
8.92580131571621
9.024348910484349
5.367896645637325
9.435845090131318
9.06315917432515
7.600465145578153
5.9068744964836055
5.445397613258918
14.755191954049657
13.15577225252741
8.229834002253147
8.261501979471754
7.229429541604467
6.04445308002427
4.947430300375877
3.6169745266506412
5.9045135518351595
4.175437018133447
13.103265306873842
7.021143994849305
4.845354126534009
6.043988978128805
5.160152361443496
5.563313254158705
6.5137477789599965
21.61600319596337
13.396590748198394
8.735918207332254
9.774470060120098
10.041856280331865
12.202332224009833
6.295856790265812
6.101693812282242
7.750778846643332
6.880695048386383
6.229411297719382
9.505242650461174
9.394033883143333
5.524411895857639
7.684444820847489
4.57070688374212
6.844307738465818
10.145213856686997
11.211077106825764
14.956208746276943
26.063109117885656
9.347073110209317
18.33322284703606
7.634303027099144
7.53236465570545
6.512966324233916
7.202987545801672

In [219]:
from lightgbm import LGBMRegressor

In [220]:
estimator = LGBMRegressor(num_leaves=31)

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40, 80, 160]
}

gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(X_train, y_train)

gbm.best_params_

{'learning_rate': 0.1, 'n_estimators': 20}

In [223]:
model = LGBMRegressor(n_estimators=20, learning_rate=0.1)

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)
    rmse = (sqrt(mean_squared_error(y_test, y_pred_test)))
    print(rmse)
    
lgbm = model

19.216559644692737
15.000487650267635
10.372153870046445
8.731784742499432
10.16924668451418
5.88683466717496
11.32433058499313
10.741737802691686
6.341313005946445
5.744533805180042
5.841779920122873
14.904443333127848
10.150428659937727
6.863342837652312
6.078493512597838
7.79528718236565
4.335766956690041
4.6575812312962634
3.733169423805298
7.098013107764753
4.1422760133667555
13.01698715234662
7.595980999048529
4.338871218476775
6.1768925739477725
5.285981959547028
6.095923310006647
5.880651952942121
22.07507607537655
11.476029341022398
9.907896005959921
9.71683087985884
7.744710103907849
11.291980047754825
6.481886163965779
6.32480514234577
7.228439517817852
6.28283897519934
5.5028714921325665
9.191110556663164
8.679775920946309
5.862415902823557
6.877943324755741
3.140277996757562
7.008177179892063
10.025545136814053
11.751223838130247
15.176084702397727
24.586281492130897
11.439387222859317
15.139722744173412
7.335323633737645
7.576453567924708
5.523034203232055
6.6140909005606

In [230]:
item = X_valid[:,0]

xgb_pred = xgb.predict(X_valid)
xgb_rmse = (sqrt(mean_squared_error(y_valid, xgb_pred)))
xgb_r2 = r2_score(y_valid, xgb_pred)

lgbm_pred = lgbm.predict(X_valid)
lgbm_rmse = (sqrt(mean_squared_error(y_valid, lgbm_pred)))
lgbm_r2 = r2_score(y_valid, lgbm_pred)

rfr_pred = rfr.predict(X_valid)
rfr_rmse = (sqrt(mean_squared_error(y_valid, rfr_pred)))
rfr_r2 = r2_score(y_valid, rfr_pred)

sgdr_pred = sgdr.predict(X_valid_scaled)
sgdr_rmse = (sqrt(mean_squared_error(y_valid, sgdr_pred)))
sgdr_r2 = r2_score(y_valid, sgdr_pred)

modelo = ('XGB', 'LGBM', 'RF', 'SGDR')
rmse = (xgb_rmse, lgbm_rmse, rfr_rmse, rfr_rmse)
re = (xgb_r2, lgbm_r2, rfr_r2, sgdr_r2)

error = pd.DataFrame({ 
    'modelo': modelo,  
    'rmse': rmse, 
    'r2': re, 
})
print(error)

result = pd.DataFrame({ 
    'id': item,  
    'real': y_valid, 
    'xgb_pred': xgb_pred, 
    'lgbm_pred': lgbm_pred,
    'rfr_pred': rfr_pred, 
    'sgdr_pred': sgdr_pred, 
})


print('___________________')
print(result)

  modelo       rmse        r2
0    XGB  23.214118  0.342878
1   LGBM  22.966845  0.342878
2     RF  25.306304  0.202188
3   SGDR  25.306304 -0.082807
___________________
    id  real   xgb_pred  lgbm_pred  rfr_pred  sgdr_pred
0   21    35  34.201164  34.238711    22.296   6.547618
1   62     1   1.415206   2.233627     2.963   2.971051
2   41     3   2.858361   6.356616     4.986   4.802951
3   38     3   2.299293   6.090168     4.282   5.064651
4   33     1   0.988426   5.218437     2.770   5.500818
..  ..   ...        ...        ...       ...        ...
95  44     1   8.347164   7.841294     4.444   5.339418
96  11     2  22.089611   6.715194     4.453   8.218118
97  21    13  79.393654  38.487518    26.051  11.225119
98  41     3   7.272833   7.414593     5.900   9.480452
99  38     2   5.109280   7.148145     5.089   9.742152

[100 rows x 6 columns]
